In [1]:
import time
import pandas as pd
import json

## --- Day 1: Report Repair ---
After saving Christmas five years in a row, you've decided to take a vacation at a nice resort on a tropical island. Surely, Christmas will go on without you.

The tropical island has its own currency and is entirely cash-only. The gold coins used there have a little picture of a starfish; the locals just call them **stars**. None of the currency exchanges seem to have heard of them, but somehow, you'll need to find **fifty of these coins** by the time you arrive so you can pay the deposit on your room.

To save your vacation, you need to get all fifty stars by December 25th.

**Collect stars by solving puzzles.** Two puzzles will be made available on each day in the Advent calendar; the second puzzle is unlocked when you complete the first. Each puzzle grants one star. Good luck!

Before you leave, the Elves in accounting just need you to fix your expense report (your puzzle input); apparently, something isn't quite adding up.

Specifically, they need you to find **the two entries that sum to 2020 and then multiply those two numbers together**.

Of course, your expense report is much larger. Find the two entries that sum to 2020; what do you get if you multiply them together?

In [2]:
start_time = time.time()

expense = pd.read_csv('input-files/input-day01.txt', header=None)
expense_list = expense[0].tolist()

n = 0
for a in expense_list:
    for b in expense_list[(n + 1):]:
        year = a + b
        if year == 2020:
            print(f'a = {a}, b = {b} and year = {year}')
            print(a * b)
    n += 1

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

a = 1387, b = 633 and year = 2020
877971
Spent time: 0.005 s


### --- Part Two ---
The Elves in accounting are thankful for your help; one of them even offers you a starfish coin they had left over from a past vacation. They offer you a second one if you can **find three numbers in your expense report that meet the same criteria**.

Using the above example again, the three entries that sum to 2020 are 979, 366, and 675. Multiplying them together produces the answer, 241861950.

In your expense report, what is the product of the three entries that sum to 2020?

In [3]:
start_time = time.time()

n = 0
for a in expense_list:
    p = 1
    for b in expense_list[(n + 1):]:
        for c in expense_list[(n + p + 1):]:
            year = a + b + c
            if year == 2020:
                print(f'a = {a}, b = {b}, c = {c} and year = {year}')
                print(a * b * c)
        p += 1
    n += 1

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

a = 867, b = 264, c = 889 and year = 2020
203481432
Spent time: 0.210 s


## --- Day 2: Password Philosophy ---
Your flight departs in a few days from the coastal airport; the easiest way down to the coast from here is via toboggan.
The shopkeeper at the North Pole Toboggan Rental Shop is having a bad day. "Something's wrong with our computers; we can't log in!" You ask if you can take a look.

Their password database seems to be a little corrupted: **some of the passwords wouldn't have been allowed** by the Official Toboggan Corporate Policy that was in effect when they were chosen.

To try to debug the problem, they have created **a list (your puzzle input) of passwords** (according to the corrupted database) and **the corporate policy when that password was set**.

Each line gives the password policy and then the password. The password policy indicates **the lowest and highest number of times a given letter must appear** for the password to be valid. For example, 1-3 a means that the password must contain a at least 1 time and at most 3 times.

How many passwords are valid according to their policies?

In [4]:
def check(password, letter, mini, maxi):
    nb = password.count(letter)
    return ((nb >= mini) & (nb <= maxi))

In [5]:
start_time = time.time()

pswd = pd.read_csv('input-files/input-day02.txt', header=None, sep=':')
pswd.columns = ['policy', 'password']

pswd[['values', 'letter']] = pswd['policy'].str.split(' ', expand=True)
pswd[['min', 'max']] = pswd['values'].str.split('-', expand=True).astype(int)
pswd = pswd[['password', 'letter', 'min', 'max']]

pswd['check'] = pswd.apply(lambda x: check(x['password'], x['letter'], x['min'], x['max']), axis=1)

print(pswd['check'].sum())

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

564
Spent time: 0.059 s


### --- Part Two ---
While it appears you validated the passwords correctly, they don't seem to be what the Official Toboggan Corporate Authentication System is expecting.
The shopkeeper suddenly realizes that he just accidentally explained the password policy rules from his old job at the sled rental place down the street! The Official Toboggan Corporate Policy actually works a little differently.

Each policy actually **describes two positions in the password**, where 1 means the first character, 2 means the second character, and so on. (Be careful; Toboggan Corporate Policies have *no concept of "index zero"!*) **Exactly one of these positions must contain the given letter.** Other occurrences of the letter are irrelevant for the purposes of policy enforcement.

Given the same example list from above:

**How many passwords are valid** according to the new interpretation of the policies?

In [6]:
def get_letter(word, position):
    return word[position]

In [7]:
start_time = time.time()

pswd['l1'] = pswd.apply(lambda x: get_letter(x['password'], x['min']), axis=1)
pswd['l2'] = pswd.apply(lambda x: get_letter(x['password'], x['max']), axis=1)
pswd['check new'] = (pswd['l1'] == pswd['letter']) ^ (pswd['l2'] == pswd['letter'])

print(pswd['check new'].sum())

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

325
Spent time: 0.068 s


## --- Day 3: Toboggan Trajectory ---
With the toboggan login problems resolved, you set off toward the airport. While travel by toboggan might be easy, it's certainly not safe: there's very minimal steering and the area is covered in trees. You'll need to see which angles will take you near the fewest trees.

Due to the local geology, trees in this area only grow on exact integer coordinates in a grid. You make a map (your puzzle input) of **the open squares (.) and trees (#)** you can see.
These aren't the only trees, though; due to something you read about once involving arboreal genetics and biome stability, the same pattern repeats to the right many times

You start on the open square (.) in the top-left corner and need to reach the bottom (below the bottom-most row on your map).

The toboggan can only follow a few specific slopes (you opted for a cheaper model that prefers rational numbers); start by counting all the trees you would encounter for the slope **right 3, down 1**
From your starting position at the top-left, check the position that is right 3 and down 1. Then, check the position that is right 3 and down 1 from there, and so on until you go past the bottom of the map.

Starting at the top-left corner of your map and following a slope of right 3 and down 1, **how many trees would you encounter?**

In [8]:
def count_trees(mapping, right, down):
    nb_trees = 0
    height = len(mapping)
    width = len(mapping[0])
    col = 0
    for lig in range(0, height, down):
        if tree_map[lig][col] == '#':
            nb_trees += 1
        col += right
        if col >= width:
            col -= width
    return nb_trees

In [9]:
start_time = time.time()

with open('input-files/input-day03.txt', 'r') as fic:
    tempo = fic.readlines()
    tree_map = [x.rstrip('\n') for x in tempo]

print(count_trees(tree_map, 3, 1))

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

205
Spent time: 0.001 s


### --- Part Two ---
Time to check the rest of the slopes - you need to minimize the probability of a sudden arboreal stop, after all.

Determine the number of trees you would encounter if, for each of the following slopes, you start at the top-left corner and traverse the map all the way to the bottom:

- Right 1, down 1.
- Right 3, down 1. (This is the slope you already checked.)
- Right 5, down 1.
- Right 7, down 1.
- Right 1, down 2.

**What do you get if you multiply together the number of trees encountered on each of the listed slopes?**

In [10]:
start_time = time.time()

print(
    count_trees(tree_map, 1, 1) * count_trees(tree_map, 3, 1) * count_trees(tree_map, 5, 1) * 
    count_trees(tree_map, 7, 1) * count_trees(tree_map, 1, 2)
)

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.6f} s')

3952146825
Spent time: 0.001086 s


## --- Day 4: Passport Processing ---
You arrive at the airport only to realize that you grabbed your North Pole Credentials instead of your passport. While these documents are extremely similar, North Pole Credentials aren't issued by a country and therefore aren't actually valid documentation for travel in most of the world.

It seems like you're not the only one having problems, though; a very long line has formed for the automatic passport scanners, and the delay could upset your travel itinerary.

Due to some questionable network security, you realize you might be able to solve both of these problems at the same time.

The automatic passport scanners are slow because they're having trouble detecting which passports have all required fields. The expected fields are as follows:

* byr (Birth Year)
* iyr (Issue Year)
* eyr (Expiration Year)
* hgt (Height)
* hcl (Hair Color)
* ecl (Eye Color)
* pid (Passport ID)
* cid (Country ID)

Passport data is validated in batch files (your puzzle input). Each passport is represented as a sequence of key:value pairs separated by spaces or newlines. Passports are separated by blank lines.

**Missing cid is fine, but missing any other field is not, so this passport is invalid.**

According to the above rules, your improved system would report 2 valid passports.

**Count the number of valid passports** - those that have **all required fields**. Treat **cid as optional**. In your batch file, how many passports are valid?

In [11]:
start_time = time.time()

with open('input-files/input-day04.txt', 'r') as fic:
    tempo = fic.read()
    tempo = tempo.split('\n\n')

ids = pd.DataFrame([x.replace('\n', ' ').replace(' ', ', ')  for x in tempo], columns=['Id'])

ids[[f'val{n}' for n in range(8)]] = ids['Id'].str.split(', ', expand=True)

fields = ['byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid']
for field in fields:
    ids[field] = 0

for idcol in fields:
    for nkey in range(8):
        ids[f'{idcol}_{nkey}' ] = ids[f'val{nkey}'].str.startswith(idcol) + 0
ids.fillna(0, inplace=True)

print(((ids[[f'{idcol}_{nkey}' for idcol in fields for nkey in range(8)]].sum(axis=1)) == 7).sum())

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

222
Spent time: 0.106 s


### --- Part Two ---
The line is moving more quickly now, but you overhear airport security talking about how passports with invalid data are getting through. Better add some data validation, quick!

You can continue to ignore the cid field, but each other field has strict rules about what values are valid for automatic validation:

* byr (Birth Year) - four digits; at least 1920 and at most 2002.
* iyr (Issue Year) - four digits; at least 2010 and at most 2020.
* eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
* hgt (Height) - a number followed by either cm or in:
  * If cm, the number must be at least 150 and at most 193.
  * If in, the number must be at least 59 and at most 76.
* hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
* ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
* pid (Passport ID) - a nine-digit number, including leading zeroes.
* cid (Country ID) - ignored, missing or not.

Your job is to count the passports where **all required fields are both present and valid** according to the above rules.

In [12]:
def check_hgt(value):
    if value.endswith('cm'):
        return ((int(value[:-2]) >= 150) & (int(value[:-2]) <= 193)) 
    if value.endswith('in'):
        return ((int(value[:-2]) >= 59) & (int(value[:-2]) <= 76)) 
    return False

In [13]:
start_time = time.time()

ids_val = ids.loc[
    (ids[[f'{idcol}_{nkey}' for idcol in fields for nkey in range(8)]].sum(axis=1)) == 7,
    ['Id'] + [f'val{n}' for n in range(8)]
]

ids_val['Id'] = ids_val['Id'].str.replace(':', '": "')
ids_val['Id'] = ids_val['Id'].str.replace(', ', '", "')
ids_val['Id'] = ids_val['Id'].apply(lambda x: '{"' + x + '"}')

description = pd.DataFrame([json.loads(x) for x in list(ids_val['Id'])])

description['check_ecl'] = description['ecl'].apply(
    lambda x: x in ['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth']
)
description['check_eyr'] = (
    (description['eyr'].astype(int) >= 2020) & (description['eyr'].astype(int) <= 2030)
)
description['check_hgt'] = description['hgt'].apply(check_hgt)
description['check_hcl'] = description['hcl'].str.match('^#[0-9a-f]{6}$') 
description['check_iyr'] = (
    (description['iyr'].astype(int) >= 2010) & (description['iyr'].astype(int) <= 2020)
)
description['check_byr'] = (
    (description['byr'].astype(int) >= 1920) & (description['byr'].astype(int) <= 2002)
)
description['check_pid'] = description['pid'].str.match('^[0-9]{9}$') 

print((description[[col for col in description.columns if col.startswith('check_')]].sum(axis=1) == 7).sum())

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

140
Spent time: 0.019 s


## --- Day 5: Binary Boarding ---
You board your plane only to discover a new problem: you dropped your boarding pass! You aren't sure which seat is yours, and all of the flight attendants are busy with the flood of people that suddenly made it through passport control.

You write a quick program to use your phone's camera to scan all of the nearby boarding passes (your puzzle input); perhaps you can find your seat through process of elimination.

Instead of zones or groups, this airline uses binary space partitioning to seat people. A seat might be specified like FBFBBFFRLR, where **F means "front", B means "back", L means "left", and R means "right"**.

The **first 7 characters** will either be F or B; these specify exactly **one of the 128 rows** on the plane (numbered **0 through 127**). Each letter tells you which half of a region the given seat is in. Start with the whole list of rows; the first letter indicates whether the seat is in the **front (0 through 63)** or the **back (64 through 127)**. The next letter indicates which half of that region the seat is in, and so on until you're left with exactly one row.

The last three characters will be either L or R; these specify exactly one of the **8 columns** of seats on the plane (numbered **0 through 7**). The same process as above proceeds again, this time with only three steps. L means to keep the lower half, while R means to keep the upper half.

Every seat also has a unique seat ID: **multiply the row by 8, then add the column**. In this example, the seat has ID 44 * 8 + 5 = 357.

As a sanity check, look through your list of boarding passes. What is the **highest seat ID** on a boarding pass?

In [14]:
def find_place(text, down, up, minimum=0, maximum=None):
    size = len(text)
    if maximum == None:
        maximum = 2**size - 1
    if size == 1:
        if text[0] == down:
            return minimum
        elif text[0] == up:
            return maximum
        else:
            return None
    else:
        cut = (maximum + minimum + 1) / 2 - 1
        if text[0] == down:
            return find_place(text[1:], down, up, minimum, cut)
        elif text[0] == up:
            return find_place(text[1:], down, up, cut + 1, maximum)
        else:
            return None

In [15]:
start_time = time.time()

with open('input-files/input-day05.txt', 'r') as fic:
    tempo = fic.readlines()
    seats = pd.DataFrame([x.strip('\n')  for x in tempo], columns=['seat_code'])

seats['raw'] = seats['seat_code'].apply(lambda x: find_place(x[:-3], 'F', 'B'))
seats['col'] = seats['seat_code'].apply(lambda x: find_place(x[-3:], 'L', 'R'))
seats['id'] = seats['raw'] * 8 + seats['col']

print(seats.max())

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

seat_code    FFFFBBBRRR
raw                 116
col                   7
id                  935
dtype: object
Spent time: 0.015 s


### --- Part Two ---
Ding! The "fasten seat belt" signs have turned on. Time to find your seat.

It's a completely full flight, so your seat should be the only missing boarding pass in your list. However, there's a catch: some of the seats at the very front and back of the plane don't exist on this aircraft, so they'll be missing from your list as well.

Your seat wasn't at the very front or back, though; **the seats with IDs +1 and -1 from yours will be in your list**.

What is the ID of your seat?

In [16]:
def find_code(number, down, up, total_length, end=''):
    tempo = down * total_length
    if number == 0:
        fill = down + end
        return tempo[:-len(fill)] + fill
    elif number == 1:
        fill = up + end
        return tempo[:-len(fill)] + fill
    else:
        rest = number % 2
        if rest == 0:
            return find_code(number // 2, down, up, total_length, down + end)
        if rest == 1:
            return find_code(number // 2, down, up, total_length, up + end)

In [17]:
start_time = time.time()

seats.sort_values(by='id', inplace=True)
seats['diffB'] = seats['id'].diff()
seats['diffF'] = seats['id'].diff(-1) * -1

my_seat = seats.loc[
    (seats['raw'] > seats['raw'].min()) & (seats['raw'] < seats['raw'].max()) &
    ((seats['diffB'] > 1) | (seats['diffF'] > 1))
].copy()

my_id = my_seat['id'].mean()
print(my_id)

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

my_col = my_id % 8
my_raw = my_id // 8

print(find_code(my_raw, 'F', 'B', 7) + find_code(my_col, 'L', 'R', 3))

743.0
Spent time: 0.006 s
BFBBBFFRRR


## --- Day 6: Custom Customs ---
As your flight approaches the regional airport where you'll switch to a much larger plane, customs declaration forms are distributed to the passengers.

The form asks **a series of 26 yes-or-no questions marked a through z**. All you need to do is identify **the questions for which anyone in your group answers "yes"**. Since your group is just you, this doesn't take very long.

For each group, count the number of questions to which anyone answered "yes". What is the sum of those counts?

In [18]:
start_time = time.time()

with open('input-files/input-day06.txt', 'r') as fic:
    tempo = fic.read().strip('\n')
    groups = [x.split('\n') for x in tempo.split('\n\n')]
    answers = [x.replace('\n', ' ') for x in tempo.split('\n\n')]

summary = pd.DataFrame(answers, columns=['global'])

for letter in [chr(x) for x in range(ord('a'),ord('z')+1)]:
    summary[letter] = (summary['global'].str.contains(letter))

summary['sum'] = summary.sum(axis=1)

print(summary['sum'].sum())

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

7120
Spent time: 0.041 s


### --- Part Two ---
As you finish the last group's customs declaration, you notice that you misread one word in the instructions:

You don't need to identify the questions to which anyone answered "yes"; you need to **identify the questions to which everyone answered "yes"!**

For each group, count the number of questions to which everyone answered "yes". What is the sum of those counts?

In [19]:
def everyone(group, letter):
    return sum([letter in answer for answer in group]) == len(group)   

In [20]:
start_time = time.time()

total = [0 for x in range(len(groups))]
for letter in [chr(x) for x in range(ord('a'),ord('z')+1)]:
    total = [x + y for x, y in zip(total, [everyone(group, letter) for group in groups])] 

print(sum(total))

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

3570
Spent time: 0.014 s


## --- Day 7: Handy Haversacks ---
You land at the regional airport in time for your next flight. In fact, it looks like you'll even have time to grab some food: all flights are currently delayed due to issues in luggage processing.

Due to recent aviation regulations, many rules (your puzzle input) are being enforced about bags and their contents; bags must be color-coded and must contain specific quantities of other color-coded bags. Apparently, nobody responsible for these regulations considered how long they would take to enforce!

You have a **shiny gold bag**. If you wanted to carry it in at least one other bag, how many different bag colors would be valid for the outermost bag? (In other words: how many colors can, eventually, contain at least one shiny gold bag?)

So, in this example, the number of bag colors that can eventually contain at least one shiny gold bag is 4.

How many bag colors **can eventually contain at least one *shiny gold* bag?** 

In [21]:
def bag_contains_one_color(rule, colors):
    return any([color in rule for color in colors])

In [22]:
start_time = time.time()

with open('input-files/input-day07.txt', 'r') as fic:
    rules = pd.DataFrame(fic.readlines(), columns=['rule'])

rules[['color', 'containing']] = rules['rule'].str.split(' bags contain ', expand=True)

ok_bags = list(['shiny gold'])
rest_bags = rules.copy()

add_bags = [None]
while len(add_bags) > 0:
    add_bags = list(rest_bags.loc[
        rest_bags['containing'].apply(lambda rule: bag_contains_one_color(rule, ok_bags)), 
        'color'
    ])
    ok_bags = ok_bags + add_bags
    rest_bags = rest_bags.loc[rest_bags['color'].isin(ok_bags) == False]

print(len(ok_bags) - 1)

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

177
Spent time: 0.100 s


### --- Part Two ---
It's getting pretty expensive to fly these days - not because of ticket prices, but because of the ridiculous number of bags you need to buy!

Consider again your shiny gold bag and the rules from the above example:

faded blue bags contain 0 other bags.
dotted black bags contain 0 other bags.
vibrant plum bags contain 11 other bags: 5 faded blue bags and 6 dotted black bags.
dark olive bags contain 7 other bags: 3 faded blue bags and 4 dotted black bags.
So, a single shiny gold bag must contain 1 dark olive bag (and the 7 bags within it) plus 2 vibrant plum bags (and the 11 bags within each of those): 1 + 1*7 + 2 + 2*11 = 32 bags!

Of course, the actual rules have a small chance of going several levels deeper than this example; be sure to count all of the bags, even if the nesting becomes topologically impractical!

Here's another example:

shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags.
In this example, a single shiny gold bag must contain 126 other bags.

How many individual bags are required inside your single shiny gold bag?

* no other --> 0
* n de couleur "no other" + p de couleur "no other" --> n + p
* q de couleur contenant n --> q * (1 + n * ce qu'il contient)

In [23]:
def nb_bags(rule, color):
    end = rule.find(color) - 1
    start = end - 1
    try:
        return int(rule[start:end])
    except ValueError:
        return 0

In [24]:
def incremental_containing(color, rules, already_contains=0):
    rule = str(rules.loc[rules['color'] == color, 'containing'].values)[2:-4]
    if 'no other' in rule:
        already_contains
    else:
        all_colors = list(rules['color'])
        for color in all_colors:
            nb_color = nb_bags(rule, color)
            if nb_color > 0:
                local_rule = rules.loc[rules['color'] == color, 'containing']
                add = nb_color * incremental_containing(color, rules, 1)
                already_contains += add
    return already_contains

In [25]:
start_time = time.time()

print(incremental_containing('shiny gold', rules))

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

34988
Spent time: 0.154 s


## --- Day 8: Handheld Halting ---
Your flight to the major airline hub reaches cruising altitude without incident. While you consider checking the in-flight menu for one of those drinks that come with a little umbrella, you are interrupted by the kid sitting next to you.

Their handheld game console won't turn on! They ask if you can take a look.

You narrow the problem down to a strange infinite loop in the boot code (your puzzle input) of the device. You should be able to fix it, but first you need to be able to run the code in isolation.

The boot code is represented as a text file with one instruction per line of text. Each instruction consists of an operation (acc, jmp, or nop) and an argument (a signed number like +4 or -20).

* **acc increases or decreases a single global value called the accumulator** by the value given in the argument. For example, acc +7 would increase the accumulator by 7. The accumulator **starts at 0**. After an acc instruction, the **instruction immediately below** it is executed next.
* **jmp jumps to a new instruction relative** to itself. The next instruction to execute is found using the argument as an offset from the jmp instruction; for example, jmp +2 would skip the next instruction, jmp +1 would continue to the instruction immediately below it, and jmp -20 would cause the instruction 20 lines above to be executed next.
* **nop stands for No OPeration** - it does nothing. The instruction **immediately below** it is executed next.
* The moment the program tries to **run any instruction a second time**, you know it will never terminate.

Run your copy of the boot code. Immediately before any instruction is executed a second time, what value is in the accumulator?

In [26]:
def play(instructions):
    [index, acc, condition] = [0, 0, True]
    while condition:
        instructions.iloc[index, 3] = True
        
        if instructions.iloc[index]['command'] == 'acc':
            acc += instructions.iloc[index]['value']
            index += 1
        elif instructions.iloc[index]['command'] == 'jmp':
            index += instructions.iloc[index]['value']
        elif instructions.iloc[index]['command'] == 'nop':
            index += 1
            
        if index >= instructions.shape[0]:
            condition = False
        elif instructions.iloc[index]['yet']:
            condition = False
            
    return [acc, index]

In [27]:
start_time = time.time()

with open('input-files/input-day08.txt', 'r') as fic:
    instructions = pd.DataFrame(fic.read().splitlines(), columns=['instruction'])

instructions[['command', 'value']] = instructions['instruction'].str.split(expand=True)
instructions['value'] = instructions['value'].astype(int)
instructions['yet'] = False

print(play(instructions))

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

[1832, 452]
Spent time: 0.156 s


### --- Part Two ---
After some careful analysis, you believe that exactly one instruction is corrupted.

Somewhere in the program, either a jmp is supposed to be a nop, or a nop is supposed to be a jmp. (No acc instructions were harmed in the corruption of this boot code.)

The program is supposed to terminate by attempting to execute an instruction immediately after the last instruction in the file. **By changing exactly one jmp or nop**, you can repair the boot code and make it terminate correctly.

**Fix the program** so that it terminates normally by changing exactly one jmp (to nop) or nop (to jmp). What is the **value of the accumulator** after the program terminates?

In [28]:
start_time = time.time()

instructions['yet'] = False

length = instructions.shape[0]
changed = 0
end = 0
changing = {'nop': 'jmp', 'jmp': 'nop'}
while end < length:
    changed_instructions = instructions.copy()
    if changed_instructions.iloc[changed, 1] != 'acc':
        old = changed_instructions.iloc[changed, 1]
        new = changing[old]
        changed_instructions.iloc[changed, 1] = new
        [acc, end] = play(changed_instructions)
    changed += 1

print(f'change {changed - 1} index {old} to {new}')
print(f'at the end (line {end}) - acc = {acc}')

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

change 359 index jmp to nop
at the end (line 626) - acc = 662
Spent time: 27.157 s


## --- Day 9: Encoding Error ---
With your neighbor happily enjoying their video game, you turn your attention to an open data port on the little screen in the seat in front of you.

Though the port is non-standard, you manage to connect it to your computer through the clever use of several paperclips. Upon connection, the port outputs a series of numbers (your puzzle input).

The data appears to be encrypted with the **eXchange-Masking Addition System (XMAS)** which, conveniently for you, is an old cypher with an important weakness.

XMAS starts by transmitting **a preamble of 25 numbers**. After that, **each number you receive should be the sum of any two of the 25 immediately previous numbers**. The two numbers will have **different values**, and there might be more than one such pair.

The first step of attacking the weakness in the XMAS data is to find the **first number** in the list (after the preamble) which is **not the sum of two of the 25 numbers before it**. What is the first number that does not have this property?

In [29]:
def valid_number(preamble, number):
    return number in set([(x + y) for x in preamble for y in preamble if x != y])

In [30]:
start_time = time.time()

with open('input-files/input-day09.txt', 'r') as fic:
    xmas = [int(x) for x in fic.read().splitlines()]

n = 0
ok = True
while ok:
    ok = valid_number([xmas[x] for x in range(n, n + 25)], xmas[n + 25])
    n += 1
print(n)
print([xmas[x] for x in range(n - 1, n + 24)])
my_number = xmas[n + 24]
print(my_number)

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

538
[111933636, 87705418, 81082066, 86930092, 91859416, 88233680, 122513638, 135274533, 162423980, 207920006, 111382249, 102802531, 131711038, 108268393, 116729912, 246266476, 142367529, 185367491, 131634216, 134646919, 370343986, 149862429, 195965535, 168012158, 204963592]
144381670
Spent time: 0.048 s


### --- Part Two ---
The final step in breaking the XMAS encryption relies on the invalid number you just found: you must find a **contiguous set of *at least* two numbers** in your list which sum to the invalid number from step 1.

To find the encryption weakness, **add together the *smallest* and *largest* number in this contiguous range**.

What is the encryption weakness in your XMAS-encrypted list of numbers?

In [31]:
start_time = time.time()

df = pd.DataFrame(xmas, columns=[1])
total = 0
n = 0
while total != my_number:
    dfsum = df.iloc[n:].cumsum()
    dfsum = dfsum.loc[dfsum[1] <= my_number]
    total = dfsum.iloc[-1].values
    n += 1

adding_numbers = df.iloc[dfsum.index]

print((adding_numbers.min() + adding_numbers.max()).values[0])

spent_time = time.time() - start_time
print(f'Spent time: {spent_time:.3f} s')

20532569
Spent time: 0.559 s
